In [1]:
! pip install pycaret

from pycaret import classification
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
from google.colab import files
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)

'''
학습 데이터 (user_id : 10000 ~ 24999, 15000명)

train_err_data.csv : 시스템에 발생한 에러 로그
train_quality_data.csv : 시스템 퀄리티 로그
train_problem_data.csv : 사용자 불만 및 불만이 접수된 시간


테스트 데이터(user_id : 30000 ~ 44998, 14999명)

test_err_data.csv : 시스템에 발생한 에러 로그
test_quality_data.csv : 시스템 퀄리티 로그
sample_submission.csv : 사용자 불만 확률(0~1) (제출용)
'''

     |████████████████████████████████| 256kB 8.4MB/s 
     |████████████████████████████████| 6.8MB 13.0MB/s 
     |████████████████████████████████| 245kB 46.4MB/s 
     |████████████████████████████████| 1.8MB 44.7MB/s 
     |████████████████████████████████| 157.5MB 78kB/s 
     |████████████████████████████████| 174kB 27.9MB/s 
     |████████████████████████████████| 1.6MB 48.9MB/s 
     |████████████████████████████████| 102kB 8.0MB/s 
     |████████████████████████████████| 276kB 32.3MB/s 
     |████████████████████████████████| 65.8MB 122kB/s 
     |████████████████████████████████| 14.2MB 46.0MB/s 
     |████████████████████████████████| 604kB 38.6MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 81kB 7.4MB/s 
     |████████████████████████████████| 3.1MB 48.9MB/s 
     |████████████████████████████████| 81kB 6.4MB/s 
     |████████████████████████████████| 2.1MB 40.1MB/s 
     |████████████████████████████████| 81kB 7.3MB/s 
 

'\n학습 데이터 (user_id : 10000 ~ 24999, 15000명)\n\ntrain_err_data.csv : 시스템에 발생한 에러 로그\ntrain_quality_data.csv : 시스템 퀄리티 로그\ntrain_problem_data.csv : 사용자 불만 및 불만이 접수된 시간\n\n\n테스트 데이터(user_id : 30000 ~ 44998, 14999명)\n\ntest_err_data.csv : 시스템에 발생한 에러 로그\ntest_quality_data.csv : 시스템 퀄리티 로그\nsample_submission.csv : 사용자 불만 확률(0~1) (제출용)\n'

In [2]:
from google.colab import drive
drive.mount("/content/drive")
PATH = "/content/drive/MyDrive/Dacon_해석가능_모델/data/"

Mounted at /content/drive


In [3]:
train_quality = pd.read_csv(PATH + "train_quality_data.csv")
display(train_quality.head())

test_quality = pd.read_csv(PATH + "test_quality_data.csv")
display(test_quality.head())

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
2,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
3,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
4,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0


In [4]:
print(train_quality.dtypes, "\n")
print(test_quality.dtypes)

time            int64
user_id         int64
fwver          object
quality_0     float64
quality_1       int64
quality_2     float64
quality_3       int64
quality_4       int64
quality_5      object
quality_6       int64
quality_7      object
quality_8      object
quality_9      object
quality_10     object
quality_11      int64
quality_12      int64
dtype: object 

time            int64
user_id         int64
fwver          object
quality_0     float64
quality_1      object
quality_2     float64
quality_3       int64
quality_4       int64
quality_5      object
quality_6       int64
quality_7      object
quality_8      object
quality_9      object
quality_10     object
quality_11      int64
quality_12      int64
dtype: object


In [5]:
train_quality_columns = [col for col in train_quality.columns if 'quality' in col]
train_quality[train_quality_columns] = train_quality[train_quality_columns].fillna(9999999)
test_quality_columns = [col for col in test_quality.columns if 'quality' in col]
test_quality[test_quality_columns] = test_quality[test_quality_columns].fillna(9999999)

train_quality_object_columns = train_quality.select_dtypes("object").drop("fwver", axis=1).columns.tolist()
train_quality_float_columns = train_quality.select_dtypes("float64").columns.tolist()
print("Train object columns:", train_quality_object_columns)
print("Train float columns:", train_quality_float_columns)

# train_quality[train_quality_columns] = train_quality[train_quality_columns].apply(string2num)
# Above line does not work.
for index in train_quality_object_columns:
  train_quality[index] = train_quality[index].apply(string2num)
for index in train_quality_float_columns:
  train_quality[index] = train_quality[index].astype(int)

test_quality_object_columns = test_quality.select_dtypes("object").drop("fwver", axis=1).columns.tolist()
test_quality_float_columns = test_quality.select_dtypes("float64").columns.tolist()
print("Train object columns: ", test_quality_object_columns)
print("Train float columns: ", test_quality_float_columns)

for index in test_quality_object_columns:
  test_quality[index] = test_quality[index].apply(string2num)
for index in test_quality_float_columns:
  test_quality[index] = test_quality[index].astype(int)

train_quality[train_quality_columns] = train_quality[train_quality_columns].astype(str)
test_quality[test_quality_columns] = test_quality[test_quality_columns].astype(str)

Train object columns: ['quality_5', 'quality_7', 'quality_8', 'quality_9', 'quality_10']
Train float columns: ['quality_0', 'quality_2']
Train object columns:  ['quality_1', 'quality_5', 'quality_7', 'quality_8', 'quality_9', 'quality_10']
Train float columns:  ['quality_0', 'quality_2']


In [6]:
print(train_quality.dtypes, "\n")
print(test_quality.dtypes)

time           int64
user_id        int64
fwver         object
quality_0     object
quality_1     object
quality_2     object
quality_3     object
quality_4     object
quality_5     object
quality_6     object
quality_7     object
quality_8     object
quality_9     object
quality_10    object
quality_11    object
quality_12    object
dtype: object 

time           int64
user_id        int64
fwver         object
quality_0     object
quality_1     object
quality_2     object
quality_3     object
quality_4     object
quality_5     object
quality_6     object
quality_7     object
quality_8     object
quality_9     object
quality_10    object
quality_11    object
quality_12    object
dtype: object


In [7]:
train_quality_table = train_quality[train_quality_columns].apply(lambda x: x.value_counts()).T.stack()
display(train_quality_table.head())

test_quality_table = test_quality[test_quality_columns].apply(lambda x: x.value_counts()).T.stack()
display(test_quality_table.head())

quality_0  -1     130828.0
           0      542790.0
           1        2097.0
           10        279.0
           100         2.0
dtype: float64

quality_0  -1     127001.0
           0      505710.0
           1        2319.0
           10        102.0
           100         4.0
dtype: float64

In [8]:
train_quality_table["quality_7"].sort_values(ascending=False)[2:7]

2    3648.0
5    3456.0
6    2688.0
3    2568.0
4    2124.0
dtype: float64

In [9]:
test_quality_table["quality_3"]

0    747972.0
dtype: float64

In [10]:
for i in train_quality_columns:
  print(str(i) + "\n")
  print(train_quality_table[str(i)].sort_values(ascending=False)[0:22])
  print("\n")

quality_0

0          542790.0
9999999    144432.0
-1         130828.0
1            2097.0
2            1252.0
3             518.0
4             410.0
5             385.0
6             358.0
7             314.0
9             298.0
10            279.0
8             268.0
11            240.0
12            227.0
13            225.0
14            197.0
16            178.0
15            169.0
17            141.0
18            124.0
19            121.0
dtype: float64


quality_1

0      670270.0
-1     153649.0
1        2567.0
2        1140.0
3         391.0
4         202.0
5         134.0
6          71.0
7          50.0
8          35.0
9          26.0
10         24.0
11         12.0
12          7.0
13          6.0
14          5.0
21          4.0
20          4.0
18          4.0
170         4.0
16          3.0
17          3.0
dtype: float64


quality_2

0          632469.0
-1         144392.0
9999999     40113.0
1            2937.0
2            1073.0
3             580.0
4             455.0
5

In [ ]:
train_quality_rname = list(train_quality_table["quality_0"].sort_values(ascending=False).index)[:21]
test_quality_rname = list(test_quality_table["quality_0"].index)

print(train_quality_rname)
print(test_quality_rname)

['0', '9999999', '-1', '1', '2', '3', '4', '5', '6', '7', '9', '10', '8', '11', '12', '13', '14', '16', '15', '17', '18']
['-1', '0', '1', '10', '100', '101', '1018', '102', '10249', '103', '104', '10452', '10483', '105', '106', '10627', '107', '108', '109', '1094', '11', '110', '111', '112', '113', '114', '1143', '11482', '1149', '115', '116', '1161', '117', '11728', '118', '1189', '119', '12', '120', '1202', '1209', '121', '122', '123', '1234', '124', '125', '1256', '1258', '1259', '126', '128', '1281', '1282', '1286', '129', '13', '130', '131', '1317', '132', '1328', '133', '1333', '13332', '1335', '134', '1346', '1349', '135', '1352', '1355', '136', '137', '1372', '138', '139', '13907', '1392', '14', '140', '1402', '141', '142', '143', '144', '1445', '145', '1450', '146', '147', '148', '149', '15', '150', '1509', '151', '1517', '1519', '152', '1528', '153', '1534', '1536', '154', '156', '1566', '157', '158', '15853', '159', '16', '160', '16044', '161', '162', '163', '164', '16427',

In [ ]:
selected_quality = list(train_quality_rname)
# Only elements which are in 'test_quality_rname' as well. Preserve order
selected_quality = [x for x in selected_quality if x in test_quality_rname]
other_quality = list(set(selected_quality) ^ set(test_quality_rname))

print(selected_quality)
print(other_quality)

['0', '9999999', '-1', '1', '2', '3', '4', '5', '6', '7', '9', '10', '8', '11', '12', '13', '14', '16', '15', '17', '18']
['145', '215', '245', '732', '18357', '94', '353', '23248', '132', '52', '481', '237', '87', '13907', '126', '182', '19', '963', '227', '41', '2845', '810', '2555', '1355', '67', '207', '488', '343', '7303', '110', '1349', '432', '1286', '449', '119', '3322', '308', '943', '210', '611', '80', '219', '57', '656', '193', '704', '357', '736', '45', '4992', '205', '324', '74', '10249', '507', '575', '212', '84', '114', '71', '755', '472', '3336', '2308', '548', '1750', '5140', '120', '631', '112', '2071', '2512', '609', '3132', '1161', '332', '543', '871', '382', '4914', '458', '5083', '453', '72', '504', '338', '75', '6963', '771', '34', '1528', '269', '100', '2662', '696', '130', '860', '28572', '6047', '255', '185', '737', '542', '131', '371', '443', '390', '530', '256', '649', '3186', '2931', '6537', '354', '6745', '99', '1768', '524', '303', '81', '115', '1652', '1

In [ ]:
# Assign 0, 1, 2, ..., n to selected_errcode
selected_dictionary = {string : i for i,string in enumerate(selected_quality)}
# Assign 98 to other_errcode (98 is 'Others')
other_dictionary = {string : len(selected_quality) for string in other_quality}
# Merge together (No duplicate, therefore it does not matter)
merged_dictionary = dict(selected_dictionary, **other_dictionary)
print(merged_dictionary)

{'0': 0, '9999999': 1, '-1': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '2159': 10, '119': 10, '246': 10, '755': 10, '559': 10, '58': 10, '631': 10, '163': 10, '262': 10, '3729': 10, '5288': 10, '2011': 10, '233': 10, '2246': 10, '239': 10, '264': 10, '19': 10, '553': 10, '1963': 10, '95': 10, '147': 10, '1845': 10, '4099': 10, '169': 10, '4728': 10, '514': 10, '1536': 10, '144': 10, '56': 10, '44': 10, '1209': 10, '101': 10, '124': 10, '224': 10, '528': 10, '2248': 10, '3246': 10, '485': 10, '410': 10, '57': 10, '16782': 10, '1094': 10, '110': 10, '14': 10, '123': 10, '6442': 10, '166': 10, '172': 10, '52': 10, '220': 10, '138': 10, '3468': 10, '4849': 10, '458': 10, '443': 10, '77': 10, '549': 10, '1149': 10, '136': 10, '197': 10, '2524': 10, '237': 10, '724': 10, '12': 10, '350': 10, '1519': 10, '6409': 10, '308': 10, '145': 10, '2244': 10, '15': 10, '1732': 10, '705': 10, '8067': 10, '254': 10, '342': 10, '42': 10, '181': 10, '2145': 10, '674': 10, '63': 10, '771': 

In [ ]:
train_quality["quality_0"] = train_quality["quality_0"].map(merged_dictionary)
train_quality["quality_0"] = train_quality["quality_0"].fillna(len(selected_quality))
train_quality["quality_0"] = pd.to_numeric(train_quality["quality_0"], downcast="integer")

In [ ]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

id_quality = train_quality[['user_id','quality_'+'0']].values
quality = np.zeros((train_user_number, len(selected_quality)))

for person_idx, qual in tqdm(id_quality):
  # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
  quality[person_idx - train_user_id_min, qual - 1] += 1

print(quality)

100%|██████████| 828624/828624 [00:02<00:00, 343554.59it/s]

[[ 0.  0.  0. ...  0.  0. 24.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0. 96.]
 ...
 [ 0.  0.  0. ...  0.  0. 24.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [12]:
quality_list = [ 'quality_'+str(i) for i in range(0, 14, 1) ]


In [13]:
quality_list[2:12]

['quality_2',
 'quality_3',
 'quality_4',
 'quality_5',
 'quality_6',
 'quality_7',
 'quality_8',
 'quality_9',
 'quality_10',
 'quality_11']

In [14]:
d = {}
for i in range( 10 ):
    d[ "quality_" + str( i ) ] = quality_0

NameError: ignored

In [15]:
pd.DataFrame(d['quality_0'])

KeyError: ignored

In [ ]:
train_quality_rname = list(train_quality_table["quality_"+str(i)].sort_values(ascending=False).index)[0:10]

In [19]:

d = {}

for i in range(0, 13, 1):
  if(i == 3):
    continue
  elif (i == 4):
    continue
  train_quality_rname = list(train_quality_table["quality_"+str(i)].sort_values(ascending=False).index)[2:22]
  train_quality_rname = list(train_quality_table["quality_"+str(i)].sort_values(ascending=False).index)[0:5]
  test_quality_rname = list(test_quality_table["quality_"+str(i)].index)

  print(train_quality_rname)
  print(test_quality_rname)

  selected_quality = list(train_quality_rname)
  # Only elements which are in 'test_quality_rname' as well. Preserve order
  selected_quality = [x for x in selected_quality if x in test_quality_rname]
  other_quality = list(set(selected_quality) ^ set(test_quality_rname))

  print(selected_quality)
  print(other_quality)

  # Assign 0, 1, 2, ..., n to selected_errcode
  selected_dictionary = {string : i for i,string in enumerate(selected_quality)}
  # Assign 98 to other_errcode (98 is 'Others')
  other_dictionary = {string : len(selected_quality) for string in other_quality}
  # Merge together (No duplicate, therefore it does not matter)
  merged_dictionary = dict(selected_dictionary, **other_dictionary)
  print(merged_dictionary)

  test_quality["quality_"+str(i)] = test_quality["quality_"+str(i)].map(merged_dictionary)
  test_quality["quality_"+str(i)] = test_quality["quality_"+str(i)].fillna(len(selected_quality))
  test_quality["quality_"+str(i)] = pd.to_numeric(test_quality["quality_"+str(i)], downcast="integer")

  train_user_id_max = 24997
  test_user_id_max = 44997
 
  train_user_id_min = 10000
  test_user_id_min = 30000

  train_user_number = 15000
  test_user_number = 14999
  

  id_quality = test_quality[['user_id','quality_'+str(i)]].values
  quality = np.zeros((test_user_number, len(selected_quality)))

  for person_idx, qual in tqdm(id_quality):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    quality[person_idx - test_user_id_min, qual - 1] += 1

 
  d[ "quality_" + str( i ) ] = quality



['0', '9999999', '-1', '1', '2']
['-1', '0', '1', '10', '100', '101', '1018', '102', '10249', '103', '104', '10452', '10483', '105', '106', '10627', '107', '108', '109', '1094', '11', '110', '111', '112', '113', '114', '1143', '11482', '1149', '115', '116', '1161', '117', '11728', '118', '1189', '119', '12', '120', '1202', '1209', '121', '122', '123', '1234', '124', '125', '1256', '1258', '1259', '126', '128', '1281', '1282', '1286', '129', '13', '130', '131', '1317', '132', '1328', '133', '1333', '13332', '1335', '134', '1346', '1349', '135', '1352', '1355', '136', '137', '1372', '138', '139', '13907', '1392', '14', '140', '1402', '141', '142', '143', '144', '1445', '145', '1450', '146', '147', '148', '149', '15', '150', '1509', '151', '1517', '1519', '152', '1528', '153', '1534', '1536', '154', '156', '1566', '157', '158', '15853', '159', '16', '160', '16044', '161', '162', '163', '164', '16427', '165', '1652', '16524', '166', '16782', '168', '1680', '169', '17', '171', '172', '173',

100%|██████████| 747972/747972 [00:02<00:00, 319266.64it/s]


['0', '-1', '1', '2', '3']
['0', '1', '10', '10452', '11', '12', '13', '14', '142', '15', '156', '16', '167', '168', '169', '17', '18', '19', '2', '21', '3', '32', '4', '41', '43', '5', '6', '7', '73', '75', '8', '9', '94', '9999999']
['0', '1', '2', '3']
['6', '4', '156', '169', '168', '17', '32', '8', '14', '73', '13', '142', '12', '167', '43', '16', '15', '9999999', '11', '9', '7', '5', '10452', '94', '10', '41', '75', '21', '19', '18']
{'0': 0, '1': 1, '2': 2, '3': 3, '6': 4, '4': 4, '156': 4, '169': 4, '168': 4, '17': 4, '32': 4, '8': 4, '14': 4, '73': 4, '13': 4, '142': 4, '12': 4, '167': 4, '43': 4, '16': 4, '15': 4, '9999999': 4, '11': 4, '9': 4, '7': 4, '5': 4, '10452': 4, '94': 4, '10': 4, '41': 4, '75': 4, '21': 4, '19': 4, '18': 4}


100%|██████████| 747972/747972 [00:02<00:00, 313342.80it/s]


['0', '-1', '9999999', '1', '2']
['-1', '0', '1', '10', '100', '1008', '101', '1018', '102', '103', '104', '10483', '105', '106', '107', '108', '109', '1094', '11', '110', '111', '112', '113', '114', '1143', '1145', '1149', '115', '116', '1161', '117', '11728', '118', '1185', '1189', '119', '12', '120', '1202', '1209', '121', '12196', '122', '123', '123196', '1234', '124', '1246', '125', '1256', '1258', '1259', '126', '128', '1281', '1282', '1286', '129', '13', '130', '131', '1317', '132', '1328', '133', '1333', '13332', '1335', '134', '1346', '1349', '135', '1352', '1355', '136', '137', '1372', '138', '139', '1392', '14', '140', '1402', '1407', '141', '141816', '142', '143', '144', '1441', '1445', '145', '1450', '146', '147', '1473', '148', '148086', '1485', '149', '15', '150', '1509', '151', '15101', '1517', '1519', '152', '1528', '153', '1534', '1536', '154', '156', '1566', '157', '158', '15853', '159', '16', '160', '161', '162', '163', '164', '16427', '165', '1652', '166', '1661', 

...

  0%|          | 0/747972 [00:00<?, ?it/s]

['0', '-1', '1', '2', '3']
['-1', '0', '1', '10', '12', '17', '2', '3', '4', '5', '6', '8', '9']
['0', '-1', '1', '2', '3']
['10', '17', '5', '6', '8', '4', '12', '9']
{'0': 0, '-1': 1, '1': 2, '2': 3, '3': 4, '10': 5, '17': 5, '5': 5, '6': 5, '8': 5, '4': 5, '12': 5, '9': 5}


  0%|          | 0/747972 [00:00<?, ?it/s]

['0', '1', '2', '3', '4']
['0', '1', '10', '12', '14', '15', '16', '18', '19', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '1', '2', '3', '4']
['7', '5', '6', '10', '8', '14', '12', '16', '15', '19', '9', '18']
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '7': 5, '5': 5, '6': 5, '10': 5, '8': 5, '14': 5, '12': 5, '16': 5, '15': 5, '19': 5, '9': 5, '18': 5}


100%|██████████| 747972/747972 [00:02<00:00, 303462.61it/s]


In [20]:
for i in range(0, 13, 1):
  if (i == 3):
    continue
  elif (i == 4):
    continue
  dataTemp = pd.DataFrame(d[ "quality_" + str( i ) ])
  dataTemp.columns = ["quality_" + str( i ) + '_' + str( j ) for j in range(0, dataTemp.shape[1], 1)]
  if i == 0:
    df__ = dataTemp
  elif i == 1:
    df_ = pd.concat([df__, dataTemp], axis = 1)
  else:
    df_ = pd.concat([df_, dataTemp], axis = 1)



In [21]:
display(df_.head(100))

,quality_0_0,quality_0_1,quality_0_2,quality_0_3,quality_0_4,quality_1_0,quality_1_1,quality_1_2,quality_1_3,quality_2_0,quality_2_1,quality_2_2,quality_2_3,quality_2_4,quality_5_0,quality_5_1,quality_5_2,quality_5_3,quality_5_4,quality_6_0,quality_6_1,quality_6_2,quality_6_3,quality_6_4,quality_7_0,quality_7_1,quality_7_2,quality_7_3,quality_7_4,quality_8_0,quality_8_1,quality_8_2,quality_8_3,quality_8_4,quality_9_0,quality_9_1,quality_9_2,quality_9_3,quality_9_4,quality_10_0,quality_10_1,quality_10_2,quality_10_3,quality_10_4,quality_11_0,quality_11_1,quality_11_2,quality_11_3,quality_11_4,quality_12_0,quality_12_1,quality_12_2,quality_12_3,quality_12_4
0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,12.0,0.0,1.0,0.0,0.0,11.0,0.0,0.0,0.0,1.0,11.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,12.0
1,0.0,9.0,0.0,0.0,27.0,9.0,0.0,0.0,27.0,9.0,0.0,0.0,0.0,27.0,12.0,1.0,1.0,0.0,22.0,9.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,12.0,24.0,9.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,36.0
2,0.0,15.0,0.0,0.0,105.0,15.0,0.0,0.0,105.0,15.0,0.0,0.0,0.0,105.0,15.0,0.0,6.0,0.0,99.0,15.0,0.0,0.0,0.0,105.0,0.0,0.0,0.0,0.0,120.0,0.0,0.0,0.0,0.0,120.0,0.0,0.0,0.0,0.0,120.0,0.0,0.0,0.0,0.0,120.0,15.0,0.0,0.0,0.0,105.0,0.0,0.0,0.0,0.0,120.0
3,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,60.0,4.0,2.0,0.0,1.0,53.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,60.0,0.0,12.0,24.0,12.0,12.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,60.0
4,0.0,6.0,0.0,0.0,18.0,6.0,0.0,0.0,18.0,6.0,0.0,0.0,0.0,18.0,6.0,1.0,0.0,0.0,17.0,6.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,24.0,12.0,12.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,24.0
5,48.0,45.0,0.0,0.0,99.0,65.0,0.0,0.0,127.0,63.0,0.0,0.0,0.0,129.0,67.0,4.0,1.0,0.0,120.0,63.0,0.0,1.0,0.0,128.0,12.0,0.0,0.0,0.0,180.0,0.0,12.0,0.0,0.0,180.0,0.0,0.0,0.0,0.0,192.0,12.0,0.0,12.0,12.0,156.0,63.0,0.0,0.0,0.0,129.0,0.0,0.0,0.0,0.0,192.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,28.0,0.0,0.0,56.0,28.0,0.0,0.0,56.0,28.0,0.0,0.0,0.0,56.0,28.0,0.0,1.0,0.0,55.0,28.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0,0.0,84.0,0.0,0.0,0.0,0.0,84.0,0.0,0.0,0.0,0.0,84.0,0.0,0.0,0.0,0.0,84.0,28.0,2.0,0.0,1.0,53.0,12.0,0.0,0.0,12.0,60.0
9,0.0,3.0,0.0,0.0,33.0,3.0,0.0,0.0,33.0,3.0,0.0,0.0,0.0,33.0,5.0,0.0,1.0,0.0,30.0,3.0,0.0,0.0,1.0,32.0,0.0,0.0,12.0,0.0,24.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,12.0,12.0,12.0,3.0,0.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,36.0


In [22]:
df_.to_csv('/content/drive/MyDrive/Dacon_해석가능_모델/Dacon_해석가능모델_공유폴더/test_total_quality_4_[2:7].csv', index = False)

In [ ]:
d['quality_' + '0']

AttributeError: ignored

In [ ]:
len(set(test_quality['user_id']))

8268

In [ ]:
train_quality['user_id'].describe()

count    828624.000000
mean      17574.678315
std        4374.113554
min       10000.000000
25%       13685.000000
50%       17597.000000
75%       21423.000000
max       24997.000000
Name: user_id, dtype: float64